<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20230312.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* Customer Analysis
* Iterators

In [1]:
import pandas as pd 
import numpy as np
import plotnine as gg
from plotnine import ggplot, aes
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date, time, timezone, timedelta
import os
import re
from functools import reduce
from collections import defaultdict
from scipy import stats
from scipy.stats import norm

# Customer Analysis

In [2]:
os.listdir()

['.config',
 'match_result_test.csv',
 'bz_data_analyis.csv',
 'mall_customers.csv',
 'KPMG_VI_New_raw_data_update_final.csv',
 'app_wau.csv',
 'sample-sales-data.csv',
 'sales_vs_discount.csv',
 '1_customers_dataset.csv',
 'cookie_cats.csv',
 'test.csv',
 '1_order_payments_dataset.csv',
 'ab_experiment.csv',
 'WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv',
 'scanner_data.csv',
 'sales_data_with_stores.csv',
 'lead_conversion.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 'sales.csv',
 'ecommerce-data.csv',
 '1_orders_dataset.csv',
 'online_retail.xlsx',
 'groceries.csv',
 'mock_sales_data.csv',
 'online_retail_II.xlsx',
 'WA_Fn-UseC_-Telco-Customer-Churn.csv',
 'sample_data']

In [38]:
df = pd.read_csv('ecommerce-data.csv', encoding = 'unicode_escape')
df.columns = ['_'.join(re.split('(?=[A-Z][a-z])', a)).lower().lstrip('_') for a in df.columns]
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [28]:
df.isnull().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customerid      135080
country              0
dtype: int64

In [29]:
df[df['customerid'].isnull()]

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerid,country
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice_no    541909 non-null  object 
 1   stock_code    541909 non-null  object 
 2   description   540455 non-null  object 
 3   quantity      541909 non-null  int64  
 4   invoice_date  541909 non-null  object 
 5   unit_price    541909 non-null  float64
 6   customerid    406829 non-null  float64
 7   country       541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [39]:
df = df[df['country'] == 'United Kingdom']
df = df[df['customerid'].isnull() == False]
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [40]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'])
df['date'] = df['invoice_date'].to_numpy().astype('datetime64[D]')
df['week'] = df['date'] - df['date'].dt.weekday.astype('timedelta64[D]')
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerid,country,date,week
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-11-29
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-11-29
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29


In [46]:
df['cat'] = df['description'].str.split().str[-1]
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerid,country,date,week,cat
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-11-29,HOLDER
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,LANTERN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-11-29,HANGER
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,BOTTLE
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,HEART.


In [49]:
df['cat'].value_counts()

DESIGN            18445
HOLDER            10274
SIGN               9100
RETROSPOT          8524
BOX                8479
                  ...  
PEACH                 1
TEDDY/CAKE            1
TO                    1
SNOWFLAKE,PINK        1
CLUSTER               1
Name: cat, Length: 861, dtype: int64

In [53]:
cnt = df.groupby(['cat'])['customerid'].count().reset_index(name = 'cnt').sort_values(['cnt'], ascending = False)
cnt['ratio'] = cnt[['cnt']].apply(lambda x : round(x/sum(x)*100,2))
cnt

,cat,cnt,ratio
268,DESIGN,18445,5.10
407,HOLDER,10274,2.84
704,SIGN,9100,2.51
645,RETROSPOT,8524,2.36
120,BOX,8479,2.34
...,...,...,...
536,OL,1,0.00
525,"NECKL36""TURQUOIS",1,0.00
401,HENS+FLOWER,1,0.00
508,MONTAN,1,0.00


In [54]:
df[df['cat'] == 'DESIGN']

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerid,country,date,week,cat
171,536385,22663,JUMBO BAG DOLLY GIRL DESIGN,10,2010-12-01 09:56:00,1.95,17420.0,United Kingdom,2010-12-01,2010-11-29,DESIGN
174,536385,22662,LUNCH BAG DOLLY GIRL DESIGN,10,2010-12-01 09:56:00,1.65,17420.0,United Kingdom,2010-12-01,2010-11-29,DESIGN
255,536394,22866,HAND WARMER SCOTTY DOG DESIGN,96,2010-12-01 10:39:00,1.85,13408.0,United Kingdom,2010-12-01,2010-11-29,DESIGN
256,536394,22865,HAND WARMER OWL DESIGN,96,2010-12-01 10:39:00,1.85,13408.0,United Kingdom,2010-12-01,2010-11-29,DESIGN
259,536394,22349,DOG BOWL CHASING BALL DESIGN,12,2010-12-01 10:39:00,3.75,13408.0,United Kingdom,2010-12-01,2010-11-29,DESIGN
...,...,...,...,...,...,...,...,...,...,...,...
541243,581496,22865,HAND WARMER OWL DESIGN,24,2011-12-09 10:20:00,2.10,16558.0,United Kingdom,2011-12-09,2011-12-05,DESIGN
541660,581538,22367,CHILDRENS APRON SPACEBOY DESIGN,1,2011-12-09 11:34:00,1.95,14446.0,United Kingdom,2011-12-09,2011-12-05,DESIGN
541681,581538,22985,WRAP BILLBOARD FONTS DESIGN,25,2011-12-09 11:34:00,0.42,14446.0,United Kingdom,2011-12-09,2011-12-05,DESIGN
541849,581580,22894,TABLECLOTH RED APPLES DESIGN,2,2011-12-09 12:20:00,9.95,12748.0,United Kingdom,2011-12-09,2011-12-05,DESIGN


# Iterators

In [55]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M',
  'age': 73},
 {'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F',
  'age': 29},
 {'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M',
  'age': 51},
 {'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F',
  'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]

In [56]:
{k : eval(f'reduce(lambda acc, cur : acc + [cur["{k}"]], users, [])') for k in users[0].keys()}

{'mail': ['gregorythomas@gmail.com',
  'hintoncynthia@hotmail.com',
  'wwagner@gmail.com',
  'daniel79@gmail.com',
  'ujackson@gmail.com'],
 'name': ['Brett Holland',
  'Madison Martinez',
  'Michael Jenkins',
  'Karen Rodriguez',
  'Amber Rhodes'],
 'sex': ['M', 'F', 'M', 'F', 'F'],
 'age': [73, 29, 51, 32, 42]}

In [59]:
{k : [user[k] for user in users] for k in users[0].keys()}

{'mail': ['gregorythomas@gmail.com',
  'hintoncynthia@hotmail.com',
  'wwagner@gmail.com',
  'daniel79@gmail.com',
  'ujackson@gmail.com'],
 'name': ['Brett Holland',
  'Madison Martinez',
  'Michael Jenkins',
  'Karen Rodriguez',
  'Amber Rhodes'],
 'sex': ['M', 'F', 'M', 'F', 'F'],
 'age': [73, 29, 51, 32, 42]}